In [7]:
# this file is for training CSRNet 

In [8]:
# file ni for CSRNET and make sure pakai nama train_new.py kat new cell
# kalau ada masalah import, run code paling last
# and make sure kat train_new.py tukar csrnet.cbam to csrnet

# and afer finish training, rename file checkpoint.pth and model_best.pth untuk elak overwritten

In [9]:
import torch.nn as nn
import torch
from torchvision import models
from utils import save_net,load_net


In [10]:
## this is original CSRnet

class CSRNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CSRNet, self).__init__()
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512, 256, 128, 64]

        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat, in_channels=512, dilation=True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)

        if not load_weights:
            mod = models.vgg16(pretrained=True)
            self._initialize_weights()
            frontend_items = list(self.frontend.state_dict().items())
            mod_items = list(mod.state_dict().items())
            for i in range(len(frontend_items)):
                frontend_items[i][1].data[:] = mod_items[i][1].data[:]

    def forward(self,x):
        x = self.frontend(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)



In [11]:
#this part is to trained original csrnet with only 200 images

In [12]:
model = CSRNet()
x = torch.rand((1,3,255,255))
model(x).shape

torch.Size([1, 1, 31, 31])

In [16]:
!python trainn.py A_train.json A_val.json 0 0

/home/elicer/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
epoch 0, processed 0 samples, lr 0.0000100000
Epoch: [0][0/1540]	Time 0.779 (0.779)	Data 0.024 (0.024)	Loss 1152.3326 (1152.3326)	
Epoch: [0][500/1540]	Time 0.060 (0.089)	Data 0.013 (0.011)	Loss 21900.0000 (13336.7153)	
Epoch: [0][1000/1540]	Time 0.041 (0.089)	Data 0.007 (0.011)	Loss 859.1921 (11397.5414)	
Epoch: [0][1500/1540]	Time 0.118 (0.089)	Data 0.014 (0.01

In [28]:
# lr = 1e-3
!python train_new.py A_train.json A_val.json 0 0

epoch 0, processed 0 samples, lr 0.0001000000
Epoch: [0][0/770]	Time 0.723 (0.723)	Data 0.019 (0.019)	Loss 43.7175 (43.7175)	
Epoch: [0][110/770]	Time 0.111 (0.104)	Data 0.007 (0.012)	Loss 25.1468 (35.4748)	
Epoch: [0][220/770]	Time 0.112 (0.098)	Data 0.008 (0.012)	Loss 8.4730 (37.9350)	
Epoch: [0][330/770]	Time 0.080 (0.094)	Data 0.015 (0.011)	Loss 75.2221 (36.0262)	
Epoch: [0][440/770]	Time 0.104 (0.094)	Data 0.008 (0.011)	Loss 47.9261 (41.0132)	
Epoch: [0][550/770]	Time 0.110 (0.093)	Data 0.013 (0.011)	Loss 15.2723 (38.6207)	
Epoch: [0][660/770]	Time 0.062 (0.093)	Data 0.006 (0.011)	Loss 24.0777 (37.3026)	
 * MAE: 314.053, RMSE: 552.002, SSIM: 0.281, PSNR: 18.914
Saved best model at epoch 0 with MAE: 314.053, SSIM: 0.281, PSNR: 18.914
 * Best MAE so far: 314.053
epoch 1, processed 770 samples, lr 0.0001000000
Epoch: [1][0/770]	Time 0.035 (0.035)	Data 0.008 (0.008)	Loss 249.2339 (249.2339)	
Epoch: [1][110/770]	Time 0.113 (0.094)	Data 0.009 (0.012)	Loss 42.6726 (29.7980)	
Epoch: [1][2

In [38]:
!python train_new.py A_train.json A_val.json 0 0

epoch 0, processed 0 samples, lr 0.0001000000
Epoch: [0][0/1540]	Time 0.722 (0.722)	Data 0.015 (0.015)	Loss 35.4871 (35.4871)	
Epoch: [0][110/1540]	Time 0.084 (0.106)	Data 0.008 (0.011)	Loss 22.4877 (37.0857)	
Epoch: [0][220/1540]	Time 0.072 (0.104)	Data 0.010 (0.012)	Loss 45.0621 (37.8551)	
Epoch: [0][330/1540]	Time 0.059 (0.100)	Data 0.015 (0.012)	Loss 2.3658 (34.6500)	
Epoch: [0][440/1540]	Time 0.153 (0.100)	Data 0.012 (0.012)	Loss 97.1535 (37.6265)	
Epoch: [0][550/1540]	Time 0.108 (0.098)	Data 0.017 (0.012)	Loss 8.5363 (34.7345)	
Epoch: [0][660/1540]	Time 0.113 (0.097)	Data 0.016 (0.012)	Loss 0.8113 (33.9639)	
Epoch: [0][770/1540]	Time 0.126 (0.096)	Data 0.024 (0.012)	Loss 32.0365 (33.7487)	
Epoch: [0][880/1540]	Time 0.056 (0.095)	Data 0.016 (0.012)	Loss 86.1447 (32.9039)	
Epoch: [0][990/1540]	Time 0.060 (0.094)	Data 0.006 (0.012)	Loss 17.4273 (32.0644)	
Epoch: [0][1100/1540]	Time 0.105 (0.094)	Data 0.015 (0.012)	Loss 13.8710 (31.5158)	
Epoch: [0][1210/1540]	Time 0.110 (0.094)	Data

In [ ]:
#our best training  # lr = 1e-5

In [42]:
!python train_new.py A_train.json A_val.json 0 0

epoch 0, processed 0 samples, lr 0.0000100000
Epoch: [0][0/400]	Time 0.714 (0.714)	Data 0.010 (0.010)	Loss 19.2275 (19.2275)	
Epoch: [0][100/400]	Time 0.134 (0.114)	Data 0.095 (0.057)	Loss 59.2525 (34.5071)	
Epoch: [0][200/400]	Time 0.094 (0.111)	Data 0.072 (0.058)	Loss 9.7520 (32.2667)	
Epoch: [0][300/400]	Time 0.094 (0.108)	Data 0.075 (0.061)	Loss 9.6761 (29.9816)	
 * MAE: 157.589, RMSE: 254.095, SSIM: 0.405, PSNR: 20.493
Saved best model at epoch 0 with MAE: 157.589, SSIM: 0.405, PSNR: 20.493
 * Best MAE so far: 157.589
epoch 1, processed 400 samples, lr 0.0000100000
Epoch: [1][0/400]	Time 0.033 (0.033)	Data 0.014 (0.014)	Loss 2.4883 (2.4883)	
Epoch: [1][100/400]	Time 0.125 (0.098)	Data 0.080 (0.062)	Loss 32.9031 (17.8246)	
Epoch: [1][200/400]	Time 0.083 (0.100)	Data 0.041 (0.063)	Loss 22.6145 (18.9303)	
Epoch: [1][300/400]	Time 0.095 (0.102)	Data 0.069 (0.065)	Loss 1.2749 (19.5689)	
 * MAE: 137.635, RMSE: 274.457, SSIM: 0.430, PSNR: 20.807
Saved best model at epoch 1 with MAE: 137.

In [13]:
!python train_new.py A_train.json A_val.json 0 0

epoch 0, processed 0 samples, lr 0.0000100000
Epoch: [0][0/400]	Time 0.705 (0.705)	Data 0.017 (0.017)	Loss 145.1583 (145.1583)	
Epoch: [0][100/400]	Time 0.087 (0.110)	Data 0.048 (0.059)	Loss 90.8428 (184.3521)	
Epoch: [0][200/400]	Time 0.084 (0.108)	Data 0.071 (0.063)	Loss 22.0895 (164.5577)	
Epoch: [0][300/400]	Time 0.118 (0.106)	Data 0.083 (0.064)	Loss 77.4629 (143.4522)	
 * MAE: 151.370, RMSE: 275.333, SSIM: 0.349, PSNR: 22.534
Saved best model at epoch 0 with MAE: 151.370, SSIM: 0.349, PSNR: 22.534
 * Best MAE so far: 151.370
epoch 1, processed 400 samples, lr 0.0000100000
Epoch: [1][0/400]	Time 0.065 (0.065)	Data 0.017 (0.017)	Loss 38.6823 (38.6823)	
Epoch: [1][100/400]	Time 0.136 (0.098)	Data 0.089 (0.061)	Loss 79.7508 (95.7806)	
Epoch: [1][200/400]	Time 0.115 (0.101)	Data 0.083 (0.063)	Loss 30.7817 (82.2087)	
Epoch: [1][300/400]	Time 0.133 (0.102)	Data 0.095 (0.064)	Loss 77.9147 (87.3873)	
 * MAE: 119.531, RMSE: 237.489, SSIM: 0.345, PSNR: 22.773
Saved best model at epoch 1 with

In [35]:
import torch

ckpt = torch.load("0checkpoint.pth.tar", map_location='cpu')
print(ckpt.keys())


dict_keys(['epoch', 'arch', 'state_dict', 'best_mae', 'optimizer'])


In [6]:
import importlib
import model
importlib.reload(model)
import model
print(dir(model))

['CBAM', 'CSRNet_CBAM', 'ChannelAttention', 'SpatialAttention', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'make_layers', 'models', 'nn', 'torch']
